# Bayesian structure learning

Here we perform UQ for a GFlowNets trained to sample candidate graphs for a linear Gaussian network using PCE.

First, we import the necessary libraries.

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.special import softmax
import matplotlib.pyplot as plt
import pickle
import networkx as nx
import jax
import jax.numpy as jnp
import numpy as np
import optax
from pgmpy.models import LinearGaussianBayesianNetwork
from jax import vmap
import logging
import matplotlib
logging.getLogger('matplotlib.category').setLevel('WARNING')
import chaospy as chaos
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn import linear_model as lm
from numpy.random import default_rng
sns.set(font='Times New Roman', style = 'white')
import torch
import torch.nn as nn
import torch.nn.functional as F
from dag_gflownet.utils.factories import get_scorer
from dag_gflownet.scores.bge_score import BGeScore
from dag_gflownet.scores.priors import UniformPrior
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

We can import the 250/500 datasets used to train the GFN, and use this to find a low-dimensional embedding using PCA. We approximate this low-dimensional space with a mean-zero Gaussian. 

In [7]:
data = []

no_models = 250

for n in range(0,no_models):
    try:
        data.append(pd.read_csv(f'output/data{n}.csv', index_col=0))
    except: print(n)

prior = UniformPrior()

r_matrices = []
r_vectors = []

for n in range(0,no_models):

    var_names = ["A", "B", "C", "D", "E"]
    df = pd.DataFrame(data[n], columns=var_names)
    scorer = BGeScore(data=df, prior=prior)
    r_matrices.append(scorer.R)
    r_vectors.append(scorer.R.flatten())
r_vectors = np.array(r_vectors)

n_components = 2
pca = PCA(n_components=n_components)
scaler = StandardScaler()

r_scaled  = scaler.fit_transform(r_vectors)
r_vectors_pca = pca.fit_transform(r_scaled)

fig, ax = plt.subplots(1,1)
fig.set_size_inches(3,3)

ax.scatter(r_vectors_pca[:,0],r_vectors_pca[:,1], s = 10, color = 'k',alpha = 0.5)

sns.despine()
plt.tight_layout()
plt.show()

x_mean, y_mean = np.mean(r_vectors_pca,0)
Sigma = np.cov(r_vectors_pca.T)
print('Gaussian approximation of latent space')
print(x_mean, y_mean)
print(np.diag(np.diag(Sigma))) # keep only diagonal covariance (ind.)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


IndexError: list index out of range

We import the *true* underlying graph for the linear Gaussian network,

In [8]:
with open("graph.pkl", "rb") as f:
    graph = pickle.load(f)

and the 250 GFNs in the ensemble.

In [4]:
models = []
params = []
for n in range(0,250):

    prior = UniformPrior()
    scorer = BGeScore(data=data[n], prior=prior)
    env = GFlowNetDAGEnv(num_envs=1, scorer=scorer)
    replay = ReplayBuffer(capacity=1, num_variables=env.num_variables)
    gflownet = DAGGFlowNet()
    optimizer = optax.adam(1e-4)
    key = jax.random.PRNGKey(42)
    params_scaffold, state = gflownet.init(
        key,
        optimizer,
        replay.dummy['adjacency'],
        replay.dummy['mask']
    )
    loaded_data = io.load('output/model'+ str(n) + '.npz')
    loaded_online_params = loaded_data['params']
    reconstructed_params = params_scaffold._replace(online=loaded_online_params)

    params.append(reconstructed_params)
    models.append(gflownet)

NameError: name 'data' is not defined

We construct a 'trajectory' in this case which puts together the ground truth graph, and converts it into a list of actions.

In [ ]:
def get_policy_distribution(gflownet_agent, params, observations):
    log_pi = vmap(gflownet_agent.model.apply, in_axes=(None, 0, 0))(
        params, observations['adjacency'], observations['mask']
    )
    return jnp.exp(log_pi)

def sample_action_manually(gflownet_agent, params, key, observations):
    policy_probs = get_policy_distribution(gflownet_agent, params, observations)[0]

    mask = observations['mask'][0].flatten()

    full_mask = jnp.concatenate([mask, jnp.array([1.])])
    
    masked_probs = policy_probs * full_mask
    final_probs = masked_probs / jnp.sum(masked_probs)

    action = jax.random.choice(key, a=len(final_probs), p=final_probs)
    return jnp.array([action])

def action_to_edge(action):
    source = action // num_variables
    target = action % num_variables
    return (source,target)

def edge_to_actions(source,target,num_variables=5):return source*num_variables+target

list_of_edges = graph.edges()

num_variables = len(graph.nodes())

node_to_int = {node: i for i, node in enumerate(graph.nodes())}

action_sequence = [
    node_to_int[source] * num_variables + node_to_int[target]
    for source, target in list_of_edges
]

stop_action = num_variables * num_variables
action_sequence.append(stop_action)

print(f"Ground-Truth Graph: {list_of_edges}")
print(f"Actions: {action_sequence}")

From each of the 250 models, we extract the policy along this trajectory. We also use the logit transform to turn probabilities into real-numbers which can modelled by polynomials. 

In [ ]:
policies = []

for j, model in enumerate(models):

    policy_trajectory = []
    observations = env.reset()

    for i, action in enumerate(action_sequence):
        current_policy = get_policy_distribution(
            model, params[j].online, observations
        )[0]
        
        policy_trajectory.append(np.array(current_policy,dtype=np.float64))

        observations, _, _, _ = env.step(np.array([action]))
    
    policies.append(policy_trajectory)

policies = np.array(policies)

def logit(y):
    return np.log(y/(1-y))

def sigmoid(x):
    return 1/(1+np.exp(-x))

epsilon = 1e-10
A_safe = np.where(policies == 0, epsilon, policies)
logit_ensemble_policy = logit(A_safe)

Next, we construct an orthonormal polynomial basis, with respect to the Gaussian distribution.

In [ ]:
degree = 7

q0, q1 = chaos.numpoly.variable(2)

hermite_1 = chaos.expansion.hermite(degree, mu=0, sigma=np.sqrt(Sigma[0,0]),normed=False)
hermite_2 = chaos.expansion.hermite(degree, mu=0, sigma=np.sqrt(Sigma[1,1]),normed=False)

hermite1 = hermite_1(q0)
hermite2 = hermite_2(q1)

outer_product_basis = chaos.outer(hermite1, hermite2).flatten()
polynomial_basis = outer_product_basis[np.sum(outer_product_basis.exponents, axis=1) <= degree]

We then fit the coefficients using ridge regression.

In [ ]:
polynomial_models = []
all_zeros = []

for i in range(0,logit_ensemble_policy.shape[1]):

    models_step_i = []

    for j in range(0,logit_ensemble_policy.shape[2]):
        fitted_polynomial = chaos.fit_regression(
                polynomial_basis, r_vectors_pca.T, logit_ensemble_policy[:,i,j], model=lm.Ridge(fit_intercept=False))
        if np.all(policies[:,i,j]==0):
            all_zeros.append((i,j))

        models_step_i.append(fitted_polynomial)

    polynomial_models.append(models_step_i)

To generate new outputs from our surrogate model, we have to sample new datasets from the linear Gaussian network, project these into the latent space using PCA, then plug these inputs into the PCE. Model outputs are then transformed into probability distributions with the softmax.

In [ ]:
def sample_from_linear_gaussian(model, num_samples, rng=None):
    if not isinstance(model, LinearGaussianBayesianNetwork):
        raise ValueError('The model must be an instance of LinearGaussianBayesianNetwork')
    
    if rng is None:
        rng = default_rng()
    
    samples = pd.DataFrame(index=range(num_samples), columns=list(model.nodes()), dtype=float)
    
    for node in nx.topological_sort(model):
        cpd = model.get_cpds(node)
        
        if cpd.evidence:
            parent_values = samples[cpd.evidence].to_numpy()  # shape (num_samples, num_parents)
            intercept = cpd.beta[0]
            coefficients = np.array(cpd.beta[1:])
            mean = intercept + np.dot(parent_values, coefficients)
            samples[node] = rng.normal(loc=mean, scale=cpd.std)
        else:
            intercept = cpd.beta[0]
            mean = np.full(num_samples, intercept)
            samples[node] = rng.normal(loc=mean, scale=cpd.std)
    
    return samples

data_samples = []
data_scorers = []
r_matrices = []
r_vectors = []

prior = UniformPrior()
no_datasets = 10000
no_samples = 100

for n in range(0,no_datasets):
    samples = sample_from_linear_gaussian(graph, no_samples)
    data_samples.append(data_samples)
    
    var_names = ["A", "B", "C", "D", "E"]
    df = pd.DataFrame(samples, columns=var_names)
    scorer = BGeScore(data=df, prior=prior)
    data_scorers.append(scorer)

    r_matrices.append(scorer.R)
    r_vectors.append(scorer.R.flatten())
r_vectors = np.array(r_vectors)

n_components = 2
pca = PCA(n_components=n_components)
scaler = StandardScaler()

r_scaled  = scaler.fit_transform(r_vectors)
r_vectors_pca_inputs = pca.fit_transform(r_scaled)

outputs = np.zeros((no_datasets,7,26))
prob_output_softmax = np.zeros((no_datasets,7,26))

for i in range(0,7):
    for j in range(0,26):
        outputs[:,i,j] = polynomial_models[i][j](*r_vectors_pca_inputs.T)
        if (i,j) in all_zeros:
            outputs[:,i,j] = - np.inf
    prob_output_softmax[:,i,:] = softmax(outputs[:,i,:],axis=1)

We want to compare these outputs to a testing ensemble of models that was not used in the fitting of the PCE,

In [ ]:
test_data = []

no_models = 250

for n in range(250,250+no_models):
    try:
        test_data.append(pd.read_csv(f'output/data{n}.csv', index_col=0))
    except: print(n)

test_models = []
test_params = []

for n in range(250,500):

    prior = UniformPrior()
    scorer = BGeScore(data=test_data[n-250], prior=prior)
    env = GFlowNetDAGEnv(num_envs=1, scorer=scorer)
    replay = ReplayBuffer(capacity=1, num_variables=env.num_variables)
    gflownet = DAGGFlowNet()
    optimizer = optax.adam(1e-4)
    key = jax.random.PRNGKey(42)
    params_scaffold, state = gflownet.init(
        key,
        optimizer,
        replay.dummy['adjacency'],
        replay.dummy['mask']
    )
    loaded_data = io.load('output/model'+ str(n) + '.npz')
    loaded_online_params = loaded_data['params']
    reconstructed_params = params_scaffold._replace(online=loaded_online_params)

    test_params.append(reconstructed_params)
    test_models.append(gflownet)

and then extract the policies along this same specified trajectory.

In [ ]:
test_policies = []

for j, model in enumerate(test_models):

    policy_trajectory = []
    observations = env.reset()

    for i, action in enumerate(action_sequence):
        current_policy = get_policy_distribution(
            model, test_params[j].online, observations
        )[0]
        
        policy_trajectory.append(np.array(current_policy,dtype=np.float64))

        observations, _, _, _ = env.step(np.array([action]))
    
    test_policies.append(policy_trajectory)

test_policies = np.array(test_policies)

We can then plot the surrogate model against the testing policies.

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(18, 9))

ax[0, 0].axis("off")

for n in range(7):
    idx = n + 1
    i = idx // 4
    j = idx % 4

    data_n = test_policies[:, n, :]
    surrogate_n = prob_output_softmax[:, n, :]

    df_emp = pd.DataFrame(data_n)
    df_emp = df_emp.melt(var_name="Action", value_name="Probability")
    df_emp["Source"] = "Empirical"

    df_sur = pd.DataFrame(surrogate_n)
    df_sur = df_sur.melt(var_name="Action", value_name="Probability")
    df_sur["Source"] = "Surrogate"

    df_all = pd.concat([df_emp, df_sur], ignore_index=True)

    sns.boxplot(
        data=df_all, x="Action", y="Probability", hue="Source",
        showcaps=True, showfliers=False, dodge=True,
        palette={"Empirical": "darkorange", "Surrogate": "teal"},
        ax=ax[i, j]
    )

    ax[i, j].tick_params(axis="x", rotation=90)
    ax[i, j].set_title(f"Policy at Step {n+1}")
    ax[i, j].set_xlabel("Action Index")
    ax[i, j].set_ylabel("Probability")

    ax[i, j].axvline(action_sequence[n], color="grey", linestyle="dotted")

    if n == 0: 
        ax[i, j].legend(loc="lower right")
    else:
        ax[i, j].get_legend().remove()

sns.despine()
plt.tight_layout()
plt.show()

## Comparison to a multilayer perceptron

We compare our PCE surrogate model to a MLP.

In [ ]:
class MLPPolicy(nn.Module):
    def __init__(self, hidden_sizes=[64, 64], num_actions=26):
        super().__init__()
        layers = []
        input_dim = 2
        for h in hidden_sizes:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU())
            input_dim = h
        layers.append(nn.Linear(input_dim, num_actions))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.net(x)
        probs = F.softmax(logits, dim=-1)  
        return probs

In [ ]:
models = []

for i in range(0,7):
    print(i)
    dataset = torch.utils.data.TensorDataset(
    torch.tensor(r_vectors_pca, dtype=torch.float32),
    torch.tensor(policies[:, i, :], dtype=torch.float32)
)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    model = MLPPolicy()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(50):
        for xb, yb in dataloader:
            pred = model(xb)               # [batch, 26]
            loss = F.kl_div(pred.log(), yb, reduction="batchmean")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"epoch {epoch}, loss = {loss.item():.4f}")
    
    models.append(model)

We then evaluate it on the same inputs as the PCE surrogate model,

In [ ]:
outputs = np.zeros((no_datasets, 7, 26), dtype=np.float32)

for i in range(0,7):
    model = models[i]
    with torch.no_grad():
        preds = model(torch.tensor(r_vectors_pca_inputs, dtype=torch.float32))
    outputs[:, i, :] = preds.numpy()

and plot the comparison.

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(18, 9))

ax[1, 3].axis("off")

for n in range(7):
    idx = n
    i = idx // 4
    j = idx % 4

    data_n = test_policies[:, n, :]
    mlp_n = mlp_outputs[:, n, :]
    surrogate_n = prob_output_softmax[:, n, :]

    df_emp = pd.DataFrame(data_n)
    df_emp = df_emp.melt(var_name="Action", value_name="Probability")
    df_emp["Source"] = "Empirical"

    df_mlp = pd.DataFrame(mlp_n)
    df_mlp = df_mlp.melt(var_name="Action", value_name="Probability")
    df_mlp["Source"] = "MLP"

    df_sur = pd.DataFrame(surrogate_n)
    df_sur = df_sur.melt(var_name="Action", value_name="Probability")
    df_sur["Source"] = "PCE"

    df_all = pd.concat([df_emp, df_mlp, df_sur], ignore_index=True)

    sns.boxplot(
        data=df_all, x="Action", y="Probability", hue="Source",
        showcaps=True, showfliers=False, dodge=True,
        palette={"Empirical": "darkorange", "MLP": "orchid", "PCE": "teal"},
        ax=ax[i, j]
    )

    ax[i, j].tick_params(axis="x", rotation=90)
    ax[i, j].set_title(f"Policy at Step {n+1}")
    ax[i, j].set_xlabel("Action Index")
    ax[i, j].set_ylabel("Probability")

    ax[i, j].axvline(action_sequence[n], color="grey", linestyle="dotted")

    if n == 0:
        ax[i, j].legend(loc="lower right")
    else:
        ax[i, j].get_legend().remove()

sns.despine()
plt.tight_layout()
plt.show()